# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waajid,3.8096,43.2463,31.78,39,88,9.67,SO,1690027114
1,1,aykhal,66.0000,111.5000,24.87,41,87,6.39,RU,1690027114
2,2,garachico,28.7781,-17.7646,24.06,61,20,9.77,ES,1690027114
3,3,badger,64.8000,-147.5333,15.93,85,0,2.57,US,1690027114
4,4,naze,28.3667,129.4833,27.51,85,100,3.82,JP,1690027115


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(x="Lng", y="Lat", geo=True, tiles="OSM", 
                                       frame_width=700, frame_height = 500, 
                                       size="Humidity", scale = 0.4, color="City" )

# Display the map
city_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Ideal criteria: 70F (21.1C) < Max Temp < 100F (37.8C), Humidity < 60%, Wind Speed < 10m/s 
# & in one of the following countries: Australia, Brazil, France, Germany, Hungary, Ireland, Italy, New Zealand, Peru, Portugal, South Africa, Spain, Switzerland, United Kingdom, United States
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"]>21.1) & (city_data_df["Max Temp"]<37.8) & (city_data_df["Humidity"]<60) & (city_data_df["Wind Speed"]<10) & 
                                ((city_data_df["Country"]=="AU") | (city_data_df["Country"]=="BR") | (city_data_df["Country"]=="FR") | (city_data_df["Country"]=="DE")
                                 | (city_data_df["Country"]=="HU") | (city_data_df["Country"]=="IE") | (city_data_df["Country"]=="IT") | (city_data_df["Country"]=="NZ")
                                 | (city_data_df["Country"]=="PE") | (city_data_df["Country"]=="PT") | (city_data_df["Country"]=="ZA") | (city_data_df["Country"]=="ES")
                                 | (city_data_df["Country"]=="CH") | (city_data_df["Country"]=="GB") | (city_data_df["Country"]=="US"))]

# Drop any rows with null values
ideal_cities = ideal_cities.dropna()

# Display sample data
ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,meze,43.4250,3.6059,31.09,50,2,3.45,FR,1690027119
37,37,sao felix do xingu,-6.6447,-51.9950,24.85,39,2,1.59,BR,1690026854
42,42,colonia,50.9333,6.9500,25.53,37,0,4.12,DE,1690026859
98,98,dom pedro,-4.4833,-44.4500,26.68,43,4,3.66,BR,1690027132
111,111,ipora,-16.4419,-51.1178,22.67,49,0,2.95,BR,1690027134
232,232,kununurra,-15.7667,128.7333,25.99,31,40,4.63,AU,1690027158
255,255,lahnstein,50.3000,7.6167,23.60,44,87,4.62,DE,1690027163
293,293,januaria,-15.4833,-44.3667,22.03,54,80,4.70,BR,1690027170
322,322,miracema do tocantins,-9.5618,-48.3967,26.33,43,2,6.35,BR,1690027176
340,340,acarau,-2.8856,-40.1200,28.08,49,2,5.63,BR,1690027180


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities.copy()
hotel_df = hotel_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
29,meze,FR,43.4250,3.6059,50,
37,sao felix do xingu,BR,-6.6447,-51.9950,39,
42,colonia,DE,50.9333,6.9500,37,
98,dom pedro,BR,-4.4833,-44.4500,43,
111,ipora,BR,-16.4419,-51.1178,49,
232,kununurra,AU,-15.7667,128.7333,31,
255,lahnstein,DE,50.3000,7.6167,44,
293,januaria,BR,-15.4833,-44.3667,54,
322,miracema do tocantins,BR,-9.5618,-48.3967,43,
340,acarau,BR,-2.8856,-40.1200,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel", "apiKey":geoapify_key, "limit":1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
meze - nearest hotel: Hôtel de Thau
sao felix do xingu - nearest hotel: Hotel Terraço
colonia - nearest hotel: Wasserturm Hotel Cologne
dom pedro - nearest hotel: No hotel found
ipora - nearest hotel: No hotel found
kununurra - nearest hotel: Wunan House
lahnstein - nearest hotel: Ferienwohnung Jäger
januaria - nearest hotel: No hotel found
miracema do tocantins - nearest hotel: No hotel found
acarau - nearest hotel: No hotel found
paranaiba - nearest hotel: Hotel Sul
lata - nearest hotel: Coimbra Green Hostel
senador jose porfirio - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
ferreiras - nearest hotel: Novochoro
evora - nearest hotel: Évora Olive Hotel
terra santa - nearest hotel: Hotel Novo Horizonte
santa fe do sul - nearest hotel: Litani Park Hotel
las vegas - nearest hotel: Downtown Grand
calipatria - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily


,City,Country,Lat,Lng,Humidity,Hotel Name
29,meze,FR,43.4250,3.6059,50,Hôtel de Thau
37,sao felix do xingu,BR,-6.6447,-51.9950,39,Hotel Terraço
42,colonia,DE,50.9333,6.9500,37,Wasserturm Hotel Cologne
98,dom pedro,BR,-4.4833,-44.4500,43,No hotel found
111,ipora,BR,-16.4419,-51.1178,49,No hotel found
232,kununurra,AU,-15.7667,128.7333,31,Wunan House
255,lahnstein,DE,50.3000,7.6167,44,Ferienwohnung Jäger
293,januaria,BR,-15.4833,-44.3667,54,No hotel found
322,miracema do tocantins,BR,-9.5618,-48.3967,43,No hotel found
340,acarau,BR,-2.8856,-40.1200,49,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(x="Lng", y="Lat", geo=True, tiles="OSM", 
                                    frame_width=700, frame_height = 500, 
                                    size="Humidity", scale = 0.75, color="City", 
                                    hover_cols=["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"] )

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)